In [ ]:
#Import packages 
import os
import sys

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.tri as tri
import matplotlib.colors as colors

from scipy.io import FortranFile
from scipy.interpolate import LinearNDInterpolator as Li
from scipy.interpolate import NearestNDInterpolator as Ni
from scipy.spatial import Delaunay as Dl

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import importlib
import glob


In [ ]:
Ds = xr.open_dataset( 'C540_CONUS_SCRIP_desc.nc' )

In [ ]:
lon=Ds['grid_center_lon'].values
lat=Ds['grid_center_lat'].values
crlon=Ds['grid_corner_lon'].values
crlat=Ds['grid_corner_lat'].values
refine=Ds['grid_refinement_factor'].values



In [ ]:
xlat=np.linspace(-90,90,num=181)
xlon=np.linspace(0,360,num=360)
X,Y=np.meshgrid(xlon,xlat)
print(X.shape)
Z=X*0.

ooo=np.c_[lon,lat]

dlo=Dl(ooo)

# Interpolote ocean mask to lat lon
grinx = Ni( dlo, refine )
refine_xy = grinx( X, Y )

In [ ]:
fTopo='/glade/p/cgd/amp/pel/topo/cubedata/gmted2010_modis_bedmachine-ncube3000-220518.nc'
dTopo=xr.open_dataset( fTopo )
print( list(dTopo.variables))

In [ ]:
lat3K=dTopo['lat'].values
lon3K=dTopo['lon'].values
refine_3K = grinx( lon3K, lat3K )
print(np.shape(refine_3K ))

In [ ]:
terr=dTopo['terr'].values


In [ ]:
print(refine_3K.dtype)

In [ ]:
irefine_3K = refine_3K.astype(np.int32)

In [ ]:
print(irefine_3K.dtype)

In [ ]:
terrxy=np.reshape( terr, (6,3000,3000))
latxy=np.reshape( lat3K, (6,3000,3000))
lonxy=np.reshape( lon3K, (6,3000,3000))
ref3Kxy=np.reshape( refine_3K, (6,3000,3000))


In [ ]:
plt.contourf( terrxy[5,:,:] )

In [ ]:
dTopo.dims

In [ ]:
dic = { 
    'lon':{'dims':('grid_size'), 'data':lon3K },
    'lat':{'dims':('grid_size'), 'data':lat3K },
    'refine':{'dims':('grid_size'), 'data':refine_3K },
    'irefine':{'dims':('grid_size'), 'data':irefine_3K },
    }

dRefine = xr.Dataset.from_dict(dic)

In [ ]:
print(list(dRefine.variables))
dRefine

In [ ]:
dRefine.to_netcdf('C540_CONUS_refinement_nc3000.nc')

In [ ]:
plt.tricontourf( lonxy.flatten() , latxy.flatten(), ref3Kxy.flatten() )

In [ ]:
clev=np.linspace(-1,10,num=12)
fig = plt.figure(figsize=(15,10))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180.))
ax.coastlines(color="black")

cf = ax.contourf(xlon, xlat, refine_xy ,  levels=clev  , transform=ccrs.PlateCarree() , cmap="gist_ncar")
ax.set_title( "C540-CONUS Refinement factor ")
plt.colorbar(cf,ax=ax,shrink=.5,label='Factor')